# **RAG using the Local system using llm model**

# **Required**
install ollama :-[Link](https://ollama.com/)

model install local System instruction(command):

Instruct is fine-tuned for chat/dialogue use cases.

Example:default model:

 ollama run llama3

 including the parameter:
 
ollama run llama3:70b

Pre-trained is the base model.

Example:
default model:
ollama run llama3:text


including the parameter:

ollama run llama3:70b-text


# **Required Libaries install**

In [3]:
!pip install -q langchain sentence_transformers langchain_community weaviate-client pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.5 MB/s eta 0:00:00


#**load the documents**

In [4]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader=PyPDFDirectoryLoader('/content/data/')
documents=loader.load()

In [ ]:
documents

# **Convert into the chunk**

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=1000)
docs=text_splitter.split_documents(documents)

In [ ]:
docs

# **load the Embedding model using ollama in local system(pc,laptop)**

## before the load the embedding model use this command to download model local system:ollama pull mxbai-embed-large

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
embdedding=OllamaEmbeddings(
  model='mxbai-embed-large'
)

Not working this use the following the model

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Store the data in vector database**

In [9]:
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

In [10]:
client=weaviate.Client(
    embedded_options=EmbeddedOptions()
)

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.7/weaviate-v1.23.7-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 1803


In [11]:
vectorstore=Weaviate.from_documents(
    client=client,
    documents=docs,
    embedding=embedding,
    by_text=False

)

# **create the retriver **

In [12]:
retriever=vectorstore.as_retriever()

## LangChain Expression Language (LCEL)

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [14]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3")

In [20]:
# Define prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
rag_chain=(
    {"context":retriever, "question":RunnablePassthrough()}
          | prompt
          | llm
          | StrOutputParser()
)

In [21]:
rag_chain.invoke("what is attention ?")

'Human: You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don\'t know the answer, just say that you don\'t know.\nUse two sentences maximum and keep the answer concise.\nQuestion: what is attention ?\nContext: [Document(page_content=\'Attention Visualizations\\nInput-Input Layer5\\nIt\\nis\\nin\\nthis\\nspirit\\nthat\\na\\nmajority\\nof\\nAmerican\\ngovernments\\nhave\\npassed\\nnew\\nlaws\\nsince\\n2009\\nmaking\\nthe\\nregistration\\nor\\nvoting\\nprocess\\nmore\\ndifficult\\n.\\n<EOS>\\n<pad>\\n<pad>\\n<pad>\\n<pad>\\n<pad>\\n<pad>\\nIt\\nis\\nin\\nthis\\nspirit\\nthat\\na\\nmajority\\nof\\nAmerican\\ngovernments\\nhave\\npassed\\nnew\\nlaws\\nsince\\n2009\\nmaking\\nthe\\nregistration\\nor\\nvoting\\nprocess\\nmore\\ndifficult\\n.\\n<EOS>\\n<pad>\\n<pad>\\n<pad>\\n<pad>\\n<pad>\\n<pad>\\nFigure 3: An example of the attention mechanism following long-distance dependencies in the\\nencoder self-attention 